In [50]:
import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt

csv_dir = '/home/chesetti/Repos/LearnedJoinDiskExp/experiments/results/index_stats/'
def get_dataset_from_testcase(run):
    pos = run.find("_threads")
    return run[0:pos]

dirs = [
    '/home/chesetti/Repos/LearnedJoinDiskExp/sponge/join_ssd_all',
    #'/home/chesetti/Repos/LearnedJoinDiskExp/sponge_alex/testrun',
]
index_stats = []
for dir in dirs:
 for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    json_file = builtins.open(os.path.join(dir, test_case, 'input_results', 'inner'))
    run = (json.load(json_file))
    print(run)
    for stat in run['result']['index_stats']:
       stat['num_keys'] = run['result']['num_keys']
       stat['dataset'] = get_dataset_from_testcase(test_case)
       stat['epsilon'] = stat['index_name'][-3:]
       if stat['epsilon'] == '024':
        stat['epsilon'] = '1024'
       if stat['epsilon'] == '096':
        stat['epsilon'] = '4096'
       index_stats.append(stat)
df = pd.json_normalize(index_stats)
print(df.columns)
display(df)

{'command': 'sponge/join_ssd_all/build/benchmark_runner sponge/join_ssd_all/lognormal_threads=1/input_configs/inner', 'result': {'index_stats': [{'index_build_duration': 5185372, 'index_load_duration': 4180478516, 'index_name': 'btree256', 'index_size': 12627696}, {'index_build_duration': 6723279120, 'index_load_duration': 5090919805, 'index_name': 'flatpgm256', 'index_size': 15344}, {'index_build_duration': 6852015586, 'index_load_duration': 4814797623, 'index_name': 'pgm256', 'index_size': 15688}, {'index_build_duration': 3318564509, 'index_load_duration': 4755733953, 'index_name': 'radixspline1024', 'index_size': 536878696}, {'index_build_duration': 3285232297, 'index_load_duration': 4760798186, 'index_name': 'radixspline256', 'index_size': 536886344}, {'index_build_duration': 3298255244, 'index_load_duration': 4758493592, 'index_name': 'radixspline4096', 'index_size': 536874872}, {'index_build_duration': 50092969, 'index_load_duration': 3238579358, 'index_name': 'sampledflatpgm1024

,index_build_duration,index_load_duration,index_name,index_size,num_keys,dataset,epsilon
0,5185372,4180478516,btree256,12627696,200000000,lognormal,256
1,6723279120,5090919805,flatpgm256,15344,200000000,lognormal,256
2,6852015586,4814797623,pgm256,15688,200000000,lognormal,256
3,3318564509,4755733953,radixspline1024,536878696,200000000,lognormal,1024
4,3285232297,4760798186,radixspline256,536886344,200000000,lognormal,256
...,...,...,...,...,...,...,...
65,3310658008,4762859964,radixspline4096,536873768,200000000,normal,4096
66,49634129,3210332780,sampledflatpgm1024,5536,200000000,normal,1024
67,59785615,3262587234,sampledflatpgm256,10960,200000000,normal,256
68,52073559,3244201813,sampledflatpgm4096,2768,200000000,normal,4096


In [67]:
rows = duckdb.sql(
        " SELECT index_name, dataset, index_build_duration, index_size FROM df "
        " ORDER BY index_build_duration"
    ).df()
fb_rmi = {'index_name': 'rmi', 'dataset': 'fb', 'index_build_duration': 22840080136, 'index_size': 402653200}
osm_rmi = {'index_name': 'rmi', 'dataset': 'osm', 'index_build_duration': 66079622803, 'index_size': 402653216}
books_rmi = {'index_name': 'rmi', 'dataset': 'books', 'index_build_duration': 74947767173, 'index_size': 402653216}
wiki_rmi = {'index_name': 'rmi', 'dataset': 'wiki', 'index_build_duration': 13247391814, 'index_size': 402653216}
rows.loc[len(rows)] = fb_rmi
rows.loc[len(rows)] = books_rmi
rows.loc[len(rows)] = osm_rmi
rows.loc[len(rows)] = wiki_rmi
rows['index_build_duration'] = rows['index_build_duration'] / 1e9
rows['index_size'] = rows['index_size'] /  1e6

def save_index_stats(dataset, path):
    print(dataset)
    tmp = duckdb.sql(f"SELECT index_name, dataset, index_build_duration, index_size FROM rows WHERE dataset='{dataset}' ORDER BY index_size").df()
    print(tmp.to_markdown(floatfmt='0.6f'))
    print(path)
    tmp.to_csv(os.path.join(csv_dir, f'{dataset}.txt'))

def print_plot(indexes, datasets, stat):
    print(stat)
    tmp = duckdb.sql(f"SELECT {stat}, index_name, dataset FROM rows").df();
    for index in indexes:
        plot_line = f'\\addplot[{index}] coordinates '
        plot_line += '{'
        for dataset in datasets:
            tmp = duckdb.sql(f"SELECT {stat} FROM rows WHERE index_name='{index}' AND dataset='{dataset['col']}'").df();
            value = float(tmp.loc[0][stat])
            plot_line += f"({dataset['name']}, {value}) "
            
        plot_line += '};'
        print(plot_line)
    print()

print_plot(
    ['btree256', 'pgm256', 'sampledflatpgm256', 'radixspline256'], 
    [
        {'col': 'fb', 'name': 'FB'},
        {'col': 'wiki', 'name': 'Wiki'},
        {'col': 'osm', 'name': 'OSM'},
    ], 
    'index_build_duration')

print_plot(
    ['btree256', 'pgm256', 'sampledflatpgm256', 'radixspline256'], 
    [
        {'col': 'fb', 'name': 'FB'},
        {'col': 'wiki', 'name': 'Wiki'},
        {'col': 'osm', 'name': 'OSM'},
    ], 
    'index_size')

print_plot(
    ['btree256', 'pgm256', 'sampledflatpgm256', 'radixspline256'], 
    [
        {'col': 'uniform_dense', 'name': 'udense'},
        {'col': 'uniform_sparse', 'name': 'usparse'},
        {'col': 'normal', 'name': 'lognormal'},
        {'col': 'lognormal', 'name': 'normal'},
    ], 
    'index_build_duration')

print_plot(
    ['btree256', 'pgm256', 'sampledflatpgm256', 'radixspline256'], 
    [
        {'col': 'uniform_dense', 'name': 'udense'},
        {'col': 'uniform_sparse', 'name': 'usparse'},
        {'col': 'normal', 'name': 'lognormal'},
        {'col': 'lognormal', 'name': 'normal'},
    ], 
    'index_size')

print(os.path.join(csv_dir, 'fb.txt'))
save_index_stats('fb', os.path.join(csv_dir, 'fb.txt'))

print("wiki")
tmp = duckdb.sql("SELECT index_name, dataset, index_build_duration, index_size FROM rows WHERE dataset='wiki' ORDER BY index_size").df()
print(tmp.to_markdown(floatfmt='0.3f'))

print("books")
tmp = duckdb.sql("SELECT index_name, dataset, index_build_duration, index_size FROM rows WHERE dataset='books' ORDER BY index_size").df()
print(tmp.to_markdown(floatfmt='0.3f'))

print("osm")
tmp = duckdb.sql("SELECT index_name, dataset, index_build_duration, index_size FROM rows WHERE dataset='osm' ORDER BY index_size").df()
print(tmp.to_markdown(floatfmt='0.3f'))

save_index_stats('uniform_dense', os.path.join(csv_dir, 'fb.txt'))
save_index_stats('uniform_sparse', os.path.join(csv_dir, 'fb.txt'))
save_index_stats('normal', os.path.join(csv_dir, 'fb.txt'))
save_index_stats('lognormal', os.path.join(csv_dir, 'fb.txt'))

index_build_duration
\addplot[btree256] coordinates {(FB, 0.005184973)) (Wiki, 0.002257013)) (OSM, 0.022885481)) }
\addplot[pgm256] coordinates {(FB, 5.757901595)) (Wiki, 2.152632365)) (OSM, 22.111017802)) }
\addplot[sampledflatpgm256] coordinates {(FB, 0.061837178)) (Wiki, 0.02998885)) (OSM, 0.25421726)) }
\addplot[radixspline256] coordinates {(FB, 3.714892817)) (Wiki, 1.917840776)) (OSM, 13.446359575)) }

index_size
\addplot[btree256] coordinates {(FB, 12.627696)) (Wiki, 5.714384)) (OSM, 50.498448)) }
\addplot[pgm256] coordinates {(FB, 4.14388)) (Wiki, 0.117056)) (OSM, 10.993776)) }
\addplot[sampledflatpgm256] coordinates {(FB, 3.16232)) (Wiki, 0.112112)) (OSM, 8.67352)) }
\addplot[radixspline256] coordinates {(FB, 958.3752)) (Wiki, 946.388132)) (OSM, 808.588376)) }

index_build_duration
\addplot[btree256] coordinates {(udense, 0.005253055)) (usparse, 0.005184422)) (lognormal, 0.005205875)) (normal, 0.005185372)) }
\addplot[pgm256] coordinates {(udense, 7.176311307)) (usparse, 3.5403